In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from scipy.stats import chi2
from scipy.stats import chi2_contingency
np.random.seed(10)

In [2]:
demand = pd.read_csv('price_demand_data.csv', parse_dates=[1], dayfirst=True)
display(demand)
demand.info()

,REGION,SETTLEMENTDATE,TOTALDEMAND,PRICECATEGORY
0,VIC1,2021-01-01 00:30:00,4179.21,LOW
1,VIC1,2021-01-01 01:00:00,4047.76,LOW
2,VIC1,2021-01-01 01:30:00,3934.70,LOW
3,VIC1,2021-01-01 02:00:00,3766.45,LOW
4,VIC1,2021-01-01 02:30:00,3590.37,LOW
...,...,...,...,...
11659,VIC1,2021-08-31 22:00:00,4861.91,MEDIUM
11660,VIC1,2021-08-31 22:30:00,4748.74,MEDIUM
11661,VIC1,2021-08-31 23:00:00,4620.09,MEDIUM
11662,VIC1,2021-08-31 23:30:00,4834.00,MEDIUM


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11664 entries, 0 to 11663
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   REGION          11664 non-null  object        
 1   SETTLEMENTDATE  11664 non-null  datetime64[ns]
 2   TOTALDEMAND     11664 non-null  float64       
 3   PRICECATEGORY   11664 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 364.6+ KB


In [3]:
weather = pd.read_csv('clean_weather_data.csv', parse_dates=[0], dayfirst=True)
display(weather)
weather.info()

,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),...,9am cloud amount (oktas),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,2021-01-01,15.6,29.9,0.0,2.8,9.3,NNE,31.0,13:14,19.2,...,6,N,2.0,1018.8,28.1,43,5.0,E,13,1015.3
1,2021-01-02,18.4,29.0,0.0,9.4,1.3,NNW,30.0,8:22,23.3,...,7,NNW,17.0,1013.3,28.7,38,7.0,SW,4,1008.5
2,2021-01-03,17.0,26.2,12.6,4.8,7.1,WSW,33.0,17:55,18.3,...,8,WSW,4.0,1007.7,23.5,59,4.0,SSW,2,1005.2
3,2021-01-04,16.0,18.6,2.6,3.8,0.0,SSE,41.0,16:03,16.2,...,8,SSE,11.0,1010.0,18.2,82,8.0,SSW,17,1011.0
4,2021-01-05,15.9,19.1,11.2,1.0,0.0,SSE,35.0,11:02,17.2,...,8,SSE,13.0,1012.5,18.2,82,8.0,SSE,19,1013.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2021-08-27,4.6,13.6,0.0,1.2,3.8,SSW,15.0,12:18,7.7,...,7,,0.0,1020.0,12.8,65,7.0,SSE,7,1017.4
239,2021-08-28,5.3,17.8,0.0,1.6,9.6,N,39.0,13:14,9.1,...,1,N,7.0,1018.6,17.4,31,3.0,NNW,24,1013.5
240,2021-08-29,9.1,16.2,0.6,6.4,4.3,NNE,33.0,1:50,10.6,...,7,N,13.0,1011.4,12.8,84,7.0,S,6,1010.4
241,2021-08-30,6.4,17.6,4.0,1.4,7.4,NNW,50.0,14:04,11.1,...,7,N,15.0,1016.1,16.8,45,1.0,NNW,28,1013.2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Date                               243 non-null    datetime64[ns]
 1   Minimum temperature (°C)           243 non-null    float64       
 2   Maximum temperature (°C)           243 non-null    float64       
 3   Rainfall (mm)                      243 non-null    float64       
 4   Evaporation (mm)                   243 non-null    float64       
 5   Sunshine (hours)                   243 non-null    float64       
 6   Direction of maximum wind gust     240 non-null    object        
 7   Speed of maximum wind gust (km/h)  243 non-null    float64       
 8   Time of maximum wind gust          240 non-null    object        
 9   9am Temperature (°C)               243 non-null    float64       
 10  9am relative humidity (%)          243

In [4]:
# Replace PRICECATEGORY values with Numeric values in new column PRICE_NUMERIC
demand['PRICECATEGORY'].replace( {'LOW' : 0, 'MEDIUM' : 1, 'HIGH' : 2, 'EXTREME': 3 }, inplace=True)

In [5]:
# find max TOTALDEMAND for each day
# Find max total demand value per each 48 row (per day)
demand['part'] = demand.index // 48

max_demand = demand.groupby('part')['TOTALDEMAND'].transform(max) == demand['TOTALDEMAND']

demand = demand[max_demand].reset_index()

demand['Max of TOTALDEMAND'] = demand['TOTALDEMAND']

print(demand)

     index REGION      SETTLEMENTDATE  TOTALDEMAND  PRICECATEGORY  part  \
0       34   VIC1 2021-01-01 17:30:00      5019.64              0     0   
1       81   VIC1 2021-01-02 17:00:00      4964.35              0     1   
2      132   VIC1 2021-01-03 18:30:00      4503.31              0     2   
3      180   VIC1 2021-01-04 18:30:00      4764.18              0     3   
4      225   VIC1 2021-01-05 17:00:00      4800.64              0     4   
..     ...    ...                 ...          ...            ...   ...   
238  11461   VIC1 2021-08-27 19:00:00      6769.89              2   238   
239  11509   VIC1 2021-08-28 19:00:00      5716.32              1   239   
240  11557   VIC1 2021-08-29 19:00:00      6227.89              3   240   
241  11604   VIC1 2021-08-30 18:30:00      6072.91              1   241   
242  11653   VIC1 2021-08-31 19:00:00      5779.56              1   242   

     Max of TOTALDEMAND  
0               5019.64  
1               4964.35  
2               4503.

In [6]:
'''
Extract date out of timestamp column, named new column of DataFrame as "DATE"
https://stackoverflow.com/questions/65775172/merging-pandas-dataframe-by-datetime-and-date-column
https://stackoverflow.com/questions/39662149/pandas-extract-date-and-time-from-timestamp
'''
# DATA imputation: cleaning 'Calm' 
weather = weather.replace(to_replace='Calm', value=0)

# Convert SETTLEMENTDATE Date type from 'object' to 'datetime64[ns]'
#demand['SETTLEMENTDATE'] = demand['SETTLEMENTDATE'].apply(lambda x : pd.to_datetime(str(x)))
demand['SETTLEMENTDATE'] = pd.to_datetime(demand['SETTLEMENTDATE'])

# Extract Date and Time seperately from SETTLEMENTDATE 
demand['Date'] = demand['SETTLEMENTDATE'].dt.date
demand['Time'] = demand['SETTLEMENTDATE'].dt.time

# Convert Date in 
weather['Date'] = pd.to_datetime(weather['Date']).dt.date

#demand.info()
#weather.info()

In [7]:
demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   index               243 non-null    int64         
 1   REGION              243 non-null    object        
 2   SETTLEMENTDATE      243 non-null    datetime64[ns]
 3   TOTALDEMAND         243 non-null    float64       
 4   PRICECATEGORY       243 non-null    int64         
 5   part                243 non-null    int64         
 6   Max of TOTALDEMAND  243 non-null    float64       
 7   Date                243 non-null    object        
 8   Time                243 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 17.2+ KB


In [8]:
weather.iloc[1]

Date                                 2021-01-02
Minimum temperature (°C)                   18.4
Maximum temperature (°C)                   29.0
Rainfall (mm)                               0.0
Evaporation (mm)                            9.4
Sunshine (hours)                            1.3
Direction of maximum wind gust              NNW
Speed of maximum wind gust (km/h)          30.0
Time of maximum wind gust                  8:22
9am Temperature (°C)                       23.3
9am relative humidity (%)                  52.0
9am cloud amount (oktas)                      7
9am wind direction                          NNW
9am wind speed (km/h)                      17.0
9am MSL pressure (hPa)                   1013.3
3pm Temperature (°C)                       28.7
3pm relative humidity (%)                    38
3pm cloud amount (oktas)                    7.0
3pm wind direction                           SW
3pm wind speed (km/h)                         4
3pm MSL pressure (hPa)                  

In [9]:
weather.iloc[18]

Date                                 2021-01-19
Minimum temperature (°C)                   14.2
Maximum temperature (°C)                   18.9
Rainfall (mm)                               0.0
Evaporation (mm)                            8.0
Sunshine (hours)                            9.9
Direction of maximum wind gust                S
Speed of maximum wind gust (km/h)          41.0
Time of maximum wind gust                 13:57
9am Temperature (°C)                       15.8
9am relative humidity (%)                  52.0
9am cloud amount (oktas)                      7
9am wind direction                          SSE
9am wind speed (km/h)                      15.0
9am MSL pressure (hPa)                   1023.5
3pm Temperature (°C)                       17.7
3pm relative humidity (%)                    50
3pm cloud amount (oktas)                    1.0
3pm wind direction                            S
3pm wind speed (km/h)                        19
3pm MSL pressure (hPa)                  

In [10]:
demand = demand.drop(columns = ['REGION','index','part','TOTALDEMAND','SETTLEMENTDATE'])

In [11]:
'''
export max demand result to a csv file
'''
demand.to_csv('max_totaldemand.csv',index=False)

In [12]:
demand

,PRICECATEGORY,Max of TOTALDEMAND,Date,Time
0,0,5019.64,2021-01-01,17:30:00
1,0,4964.35,2021-01-02,17:00:00
2,0,4503.31,2021-01-03,18:30:00
3,0,4764.18,2021-01-04,18:30:00
4,0,4800.64,2021-01-05,17:00:00
...,...,...,...,...
238,2,6769.89,2021-08-27,19:00:00
239,1,5716.32,2021-08-28,19:00:00
240,3,6227.89,2021-08-29,19:00:00
241,1,6072.91,2021-08-30,18:30:00


In [13]:
weather.iloc[18][0]

datetime.date(2021, 1, 19)

In [14]:
demand.iloc[18][2]

datetime.date(2021, 1, 19)

In [15]:
weather.set_index('Date')
demand.set_index('Date')
all_data = weather.merge(demand)

all_data

,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),...,9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa),PRICECATEGORY,Max of TOTALDEMAND,Time
0,2021-01-01,15.6,29.9,0.0,2.8,9.3,NNE,31.0,13:14,19.2,...,1018.8,28.1,43,5.0,E,13,1015.3,0,5019.64,17:30:00
1,2021-01-02,18.4,29.0,0.0,9.4,1.3,NNW,30.0,8:22,23.3,...,1013.3,28.7,38,7.0,SW,4,1008.5,0,4964.35,17:00:00
2,2021-01-03,17.0,26.2,12.6,4.8,7.1,WSW,33.0,17:55,18.3,...,1007.7,23.5,59,4.0,SSW,2,1005.2,0,4503.31,18:30:00
3,2021-01-04,16.0,18.6,2.6,3.8,0.0,SSE,41.0,16:03,16.2,...,1010.0,18.2,82,8.0,SSW,17,1011.0,0,4764.18,18:30:00
4,2021-01-05,15.9,19.1,11.2,1.0,0.0,SSE,35.0,11:02,17.2,...,1012.5,18.2,82,8.0,SSE,19,1013.3,0,4800.64,17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2021-08-27,4.6,13.6,0.0,1.2,3.8,SSW,15.0,12:18,7.7,...,1020.0,12.8,65,7.0,SSE,7,1017.4,2,6769.89,19:00:00
239,2021-08-28,5.3,17.8,0.0,1.6,9.6,N,39.0,13:14,9.1,...,1018.6,17.4,31,3.0,NNW,24,1013.5,1,5716.32,19:00:00
240,2021-08-29,9.1,16.2,0.6,6.4,4.3,NNE,33.0,1:50,10.6,...,1011.4,12.8,84,7.0,S,6,1010.4,3,6227.89,19:00:00
241,2021-08-30,6.4,17.6,4.0,1.4,7.4,NNW,50.0,14:04,11.1,...,1016.1,16.8,45,1.0,NNW,28,1013.2,1,6072.91,18:30:00


In [16]:
# DATA AGGREGATION
all_data = weather.merge(demand, on='Date', how='left')

# Rearrage order of features
cols = all_data.columns.tolist()
cols = cols[:1] + cols[-3:] + cols[1:-3]
all_data = all_data[cols] 
all_data

,Date,PRICECATEGORY,Max of TOTALDEMAND,Time,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,...,9am cloud amount (oktas),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,2021-01-01,0,5019.64,17:30:00,15.6,29.9,0.0,2.8,9.3,NNE,...,6,N,2.0,1018.8,28.1,43,5.0,E,13,1015.3
1,2021-01-02,0,4964.35,17:00:00,18.4,29.0,0.0,9.4,1.3,NNW,...,7,NNW,17.0,1013.3,28.7,38,7.0,SW,4,1008.5
2,2021-01-03,0,4503.31,18:30:00,17.0,26.2,12.6,4.8,7.1,WSW,...,8,WSW,4.0,1007.7,23.5,59,4.0,SSW,2,1005.2
3,2021-01-04,0,4764.18,18:30:00,16.0,18.6,2.6,3.8,0.0,SSE,...,8,SSE,11.0,1010.0,18.2,82,8.0,SSW,17,1011.0
4,2021-01-05,0,4800.64,17:00:00,15.9,19.1,11.2,1.0,0.0,SSE,...,8,SSE,13.0,1012.5,18.2,82,8.0,SSE,19,1013.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2021-08-27,2,6769.89,19:00:00,4.6,13.6,0.0,1.2,3.8,SSW,...,7,,0.0,1020.0,12.8,65,7.0,SSE,7,1017.4
239,2021-08-28,1,5716.32,19:00:00,5.3,17.8,0.0,1.6,9.6,N,...,1,N,7.0,1018.6,17.4,31,3.0,NNW,24,1013.5
240,2021-08-29,3,6227.89,19:00:00,9.1,16.2,0.6,6.4,4.3,NNE,...,7,N,13.0,1011.4,12.8,84,7.0,S,6,1010.4
241,2021-08-30,1,6072.91,18:30:00,6.4,17.6,4.0,1.4,7.4,NNW,...,7,N,15.0,1016.1,16.8,45,1.0,NNW,28,1013.2


In [20]:
'''
export all_data result to a new csv
'''
all_data.to_csv('model01_MAX_TOTALDEMAND.csv')

In [18]:
all_data.loc[[189]]

,Date,PRICECATEGORY,Max of TOTALDEMAND,Time,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,...,9am cloud amount (oktas),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
189,2021-07-09,3,7536.11,09:00:00,2.0,12.0,1.6,1.0,9.0,NNE,...,4,NE,4.0,1017.9,11.0,67,2.0,SSW,7,1015.7


In [19]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Date                               243 non-null    object 
 1   PRICECATEGORY                      243 non-null    int64  
 2   Max of TOTALDEMAND                 243 non-null    float64
 3   Time                               243 non-null    object 
 4   Minimum temperature (°C)           243 non-null    float64
 5   Maximum temperature (°C)           243 non-null    float64
 6   Rainfall (mm)                      243 non-null    float64
 7   Evaporation (mm)                   243 non-null    float64
 8   Sunshine (hours)                   243 non-null    float64
 9   Direction of maximum wind gust     240 non-null    object 
 10  Speed of maximum wind gust (km/h)  243 non-null    float64
 11  Time of maximum wind gust          240 non-null    object 